In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.preprocessing import Imputer

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import copy
import argparse

%matplotlib inline

In [2]:
nba_master = pd.read_csv('./jgrosz99-nba-player-data-1978-2016/data/nba_season_data.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18,23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
nba_master.shape

(17729, 109)

In [3]:
nba_master.head()

,year,tm,player,age,g,mp,per,ts,3par,ftr,orb,drb,trb,ast,stl,blk,tov,usg,column_s,ows,dws,ws,ws_48,column_x,obpm,dbpm,bpm,vorp,ows_48,dws_48,shot,player_id,team_mar,team_mp,team_gm,year_3par,team_ts,tm_usg,tm_ts_w_o_plyr,reb_vers,defense,val_shot,offense,mpg_int,raw_spm,min,raw_contrib,tm_sum,tm_adj,bpm_2,stderr,contrib,vorp_2,reb_vers_2,val_shot_2,offense_2,defense_2,mpg,raw_obpm,contrib_2,tm_ortg,tm_sum_2,tm_adj_2,tm_drtg,obpm_2,ostderr,ocontrib,ovorp,dbpm_2,dstderr,dcontrib,dvorp,sum_spm,min_2,mpg_2,rempg,bpm_3,contrib_3,vorp_3,vorp_gm,o_bpm,ocontrib_2,ovorp_2,ovorp_gm,d_bpm,dcontrib_2,dvorp_2,dvorp_gm,production,prod_gm,exp_bpm,truetalentbpm,exp_min,truetalentvorp,truesalary,truetimevorp,adjusted_production,worp,o_worp,d_worp,adjusted_worp,estimated_position,bbref_pos,age_on_feb_1,yrs_experience,height,weight,rounded_position,rounded_age
0,1978,NOJ,Truck Robinson,26,82,3638,15.7,0.481,NaN,0.340,8.0,25.8,17.0,6.8,0.9,1.1,13.5,23.3,NaN,1.5,5.0,6.5,0.086,NaN,-1.4,0.2,-1.2,0.7,0.020,0.066,20.2,robintr01,-1.61,19780,82,0.0,0.505,331339,0.512,-1.73,-1.12,-0.16,-0.37,2.00,-1.21,0.920,-1.12,-1.28,-0.13,-1.3,2.0,-1.2,0.6,-1.0,0.2,-0.7,-0.3,1.0,-0.8,-0.7,-1.9,1.1,-0.7,0.3,-1.5,1.3,-1.4,0.2,0.1,1.5,0.1,0.4,-1.6,0.9,44.4,42.3,-1.3,-1.2,0.6,0.6,-1.5,-1.4,0.2,0.2,0.1,0.1,0.4,0.4,"$3,900,000","$3,900,000",3.2,0.7,0.60,1.6,"$8,800,000",0.4,"$2,900,000",1.6,0.6,1.1,1.1,3.6,4.0,26.33,3.0,79.0,225.0,4.0,26.0
1,1978,BUF,Randy Smith,29,82,3314,19.1,0.521,NaN,0.326,3.6,6.4,5.0,22.2,2.4,0.2,12.8,26.8,NaN,6.1,1.7,7.9,0.114,NaN,2.8,-1.2,1.6,3.0,0.089,0.025,23.4,smithra01,-3.25,19805,82,0.0,0.518,332955,0.517,0.52,0.29,0.17,-0.19,1.54,2.16,0.837,1.81,-1.89,-0.40,1.8,2.0,1.5,3.1,0.1,0.4,1.9,0.3,0.8,3.4,2.8,-0.5,1.1,-0.3,-2.8,3.1,1.3,2.6,4.0,-1.3,1.5,-1.1,-0.8,4.4,0.8,40.4,38.5,1.8,1.5,3.1,3.1,3.1,2.6,4.0,4.0,-1.3,-1.1,-0.8,-0.8,"$16,200,000","$16,200,000",2.1,1.9,0.73,2.8,"$14,800,000",2.7,"$14,200,000",8.5,10.8,-2.3,7.4,1.0,2.0,29.14,6.0,75.0,180.0,1.0,29.0
2,1978,WSB,Elvin Hayes,32,81,3246,17.1,0.489,NaN,0.365,10.2,22.7,16.4,6.5,1.3,2.4,12.3,21.6,NaN,3.5,4.8,8.3,0.123,NaN,0.1,1.2,1.3,2.7,0.052,0.071,18.9,hayesel01,0.70,19830,82,0.0,0.506,336086,0.510,-1.31,0.08,-0.14,0.50,1.49,0.76,0.818,0.62,-2.05,0.58,1.3,2.0,1.1,2.7,-0.3,0.2,-0.2,-0.2,0.8,0.3,0.2,0.4,1.4,-0.2,0.3,0.1,1.3,0.1,1.5,1.3,1.5,1.0,1.3,-1.2,0.8,40.1,38.2,1.3,1.1,2.7,2.8,0.1,0.1,1.5,1.5,1.3,1.0,1.3,1.3,"$14,200,000","$14,400,000",2.6,1.9,0.73,2.8,"$14,700,000",2.4,"$12,700,000",7.4,3.9,3.4,6.5,4.2,4.0,32.21,9.0,81.0,235.0,4.0,32.0
3,1978,BOS,Dave Cowens,29,77,3215,18.6,0.533,NaN,0.233,8.0,25.6,17.0,15.7,1.4,1.1,13.9,18.8,NaN,6.1,4.8,10.8,0.162,NaN,1.8,2.1,4.0,4.8,0.091,0.071,16.2,cowenda01,-1.68,19830,82,0.0,0.505,332440,0.500,2.36,-0.49,0.12,1.06,1.68,4.61,0.811,3.74,-0.90,-0.22,4.4,2.0,3.6,5.2,0.4,0.4,1.2,-0.1,0.9,2.6,2.1,-1.7,1.6,-0.7,0.1,1.9,1.3,1.5,2.9,2.5,1.5,2.0,2.3,-0.6,0.8,41.8,39.7,4.4,3.6,5.2,5.5,1.9,1.5,2.9,3.1,2.5,2.0,2.3,2.4,"$26,100,000","$27,700,000",2.6,3.6,0.73,4.0,"$20,600,000",4.6,"$23,400,000",14.0,7.9,6.1,12.5,3.9,5.0,29.27,7.0,81.0,230.0,4.0,29.0
4,1978,NYK,Bob McAdoo,26,79,3182,22.0,0.567,NaN,0.412,7.8,24.7,16.4,13.4,1.4,2.0,15.8,26.1,NaN,6.9,4.0,10.9,0.165,NaN,2.4,0.8,3.1,4.1,0.104,0.061,22.0,mcadobo01,-0.47,20005,82,0.0,0.528,333391,0.518,1.37,-0.01,0.11,0.75,1.51,3.62,0.795,2.88,0.53,-0.22,3.4,2.0,2.7,4.3,0.1,0.4,1.5,-0.2,0.8,2.6,2.0,0.8,1.3,-0.1,-1.3,2.5,1.4,2.0,3.3,0.9,1.5,0.7,1.0,1.6,0.8,40.3,38.3,3.4,2.7,4.3,4.5,2.5,2.0,3.3,3.5,0.9,0.7,1.0,1.0,"$21,800,000","$22,600,000",2.5,3.0,0.73,3.6,"$18,500,000",3.9,"$19,900,000",11.6,9.0,2.6,10.6,3.6,5.0,26.36,5.0,81.0,210.0,4.0,26.0


In [7]:
for col in nba_master.columns:
    print(col)

year
tm
player
age
g
mp
per
ts
3par
ftr
orb
drb
trb
ast
stl
blk
tov
usg
column_s
ows
dws
ws
ws_48
column_x
obpm
dbpm
bpm
vorp
ows_48
dws_48
shot
player_id
team_mar
team_mp
team_gm
year_3par
team_ts
tm_usg
tm_ts_w_o_plyr
reb_vers
defense
val_shot
offense
mpg_int
raw_spm
min
raw_contrib
tm_sum
tm_adj
bpm_2
stderr
contrib
vorp_2
reb_vers_2
val_shot_2
offense_2
defense_2
mpg
raw_obpm
contrib_2
tm_ortg
tm_sum_2
tm_adj_2
tm_drtg
obpm_2
ostderr
ocontrib
ovorp
dbpm_2
dstderr
dcontrib
dvorp
sum_spm
min_2
mpg_2
rempg
bpm_3
contrib_3
vorp_3
vorp_gm
o_bpm
ocontrib_2
ovorp_2
ovorp_gm
d_bpm
dcontrib_2
dvorp_2
dvorp_gm
production
prod_gm
exp_bpm
truetalentbpm
exp_min
truetalentvorp
truesalary
truetimevorp
adjusted_production
worp
o_worp
d_worp
adjusted_worp
estimated_position
bbref_pos
age_on_feb_1
yrs_experience
height
weight
rounded_position
rounded_age


In [5]:
nba_master.sort_values(by='adjusted_worp', ascending=False)

,year,tm,player,age,g,mp,per,ts,3par,ftr,...,d_worp,adjusted_worp,estimated_position,bbref_pos,age_on_feb_1,yrs_experience,height,weight,rounded_position,rounded_age
16135,2014,GSW,Stephen Curry,27,8,270,37.4,0.710,0.547,0.333,...,1.1,35.2,1.1,1.0,27.89,6.0,75.0,185.0,1.0,28.0
16871,2016,GSW,Stephen Curry,27,8,270,37.4,0.710,0.547,0.333,...,1.1,35.2,1.1,1.0,27.89,6.0,75.0,185.0,1.0,28.0
13137,2009,CLE,LeBron James,24,81,3054,31.7,0.591,0.238,0.472,...,8.1,29.3,2.9,3.0,24.09,5.0,80.0,240.0,3.0,24.0
3761,1989,CHI,Michael Jordan,25,81,3255,31.1,0.614,0.055,0.442,...,6.8,28.5,1.9,2.0,25.96,4.0,78.0,195.0,2.0,26.0
13656,2010,CLE,LeBron James,25,76,2966,31.1,0.604,0.253,0.506,...,6.2,28.4,2.7,3.0,25.09,6.0,80.0,240.0,3.0,25.0
3381,1988,CHI,Michael Jordan,24,82,3311,31.7,0.603,0.027,0.430,...,6.0,27.7,1.8,2.0,24.96,3.0,78.0,195.0,2.0,25.0
15229,2013,MIA,LeBron James,28,76,2877,31.6,0.640,0.188,0.395,...,5.3,26.5,3.0,4.0,28.09,9.0,80.0,240.0,3.0,28.0
12618,2008,CLE,LeBron James,23,75,3027,29.1,0.568,0.219,0.470,...,5.3,25.9,2.8,3.0,23.09,4.0,80.0,240.0,3.0,23.0
13142,2009,NOH,Chris Paul,23,78,3002,30.0,0.599,0.140,0.418,...,4.9,25.8,1.0,1.0,23.74,3.0,72.0,175.0,1.0,24.0
16339,2015,OKC,Russell Westbrook,26,67,2302,29.1,0.536,0.196,0.445,...,3.9,25.5,1.1,1.0,26.22,6.0,75.0,200.0,1.0,26.0


In [6]:
nba_master.shape

(17729, 109)